# Transate cloud data to beta 

This Code translate **lwc - liquid water content* data (from a txt file) and **cross section** (from scat file according to the effective radius) into **extinction** (beta)

===

## Reading the medium txt file:
in the txt file the first 6 row will look like:

```txt
#name=Stratocumulus water cloud
#original_cloud_data=@storm/data2/gmatheou/Factory/jplles/rev153/rico.20.smagorinsky/MISR01/An
#resampled_cloud_data_grid_size=20,20,40m
1024 1024 65 3
0.020 0.020     0.000     0.040     0.080     0.120     ...     2.520     2.560     2.600
230 977 64 2.6620E-01 2.2502E+01
```

The third row specifies the grid resolution, in this example  $(dx, dy, dz) = (20, 20, 40) [m]$.

The forth row specifies the grid size, in the example medium.shape will result  $(1024, 1024, 65)$. 

And the fifth row specifies the spacing the size of x,y voxels and the z grid, meaning  $dx,\ dy,\ z_0,\ z_1,\ z_2,\ \cdots,\  z_{N_z}$

From the sixth row till the end of the file, each row specifies the data in each voxel, for example:

|   X index   |   Y index   |   Z index   |   lwc $[\frac{g}{m^3}]$   |  Effective radius $[\mu m]$  |
| :---------: | :---------: | :---------: | :-------------: | :--------------------------: |
| 230         | 977         |  64         | 2.6620E-01      |  2.2502E+01                  |


And the relation between the extinction to lwc and effective radius is,
$$extinction = lwc\cdot cross\_section(r_{eff}) = Q \frac{\pi r_{eff}^2}{\frac{4}{3}\pi {r_{eff}}^3\rho}$$

since $\rho\simeq1 [\frac{g}{cm^3}]$, we can calaulate the foloowing,
$$extinction = Q \frac{\pi r_{eff}^2}{\frac{4}{3}\pi {r_{eff}}^3}$$

---

## Reading the scat file:
in this file the first 11 row will look like:

```txt
! Mie scattering table vs. effective radius (LWC=1 g/m^3)
    0.672    0.672  wavelength range (micron)
 1.000  W   particle density (g/cm^3) and type (Water, Ice, Aerosol)
  0.133100E+01 -0.212827E-07  particle index of refraction
gamma size distribution shape parameter
  50    5.000   25.000  number, starting, ending effective radius
  50    0.010    0.200  number, starting, ending effective variance
  50   97.000    2.000  number, starting, ending shape parameter alpha
  1222  maximum number of legendre coefficients allowed
   5.0000   0.0100  0.32360E+03 0.999998    144  Reff	 Veff  Ext  Alb  Nleg
      1.00000    2.54237    3.87643    4.55551    5.22360    5.86313    6.436 ...
```

The second row specifies the wavelength it was calculated to, in this example it's $0.672 [\mu m]$.

The next row specifies the partical density in $[\frac{g}{cm^3}]$ and the types of particals. The distribution is by gamma size distribution.

From the tenth row till the end of the file specifies folowing parameters and the legendre coefficiens pf the phase function:

| Effective radius $[\mu m]$ | Effective variance | Calculated cross section |    albedo | Number of legendre coefficients |
| :------------------------: | :----------------: | :----------------------: | :-------: | :-----------------------------: |
| 5.0000                     | 0.0100             |  0.32360E+03             | 0.999998  |  144                            |

**HG phase function relation to Mie phase function**
$g = $  taking the second legendre coefficien of Mie phase function by 3

## Assumptions

* Effective varince $= 0.1031$
* Albedo $= 1$
* Wavelete $672\ [nm]$
* Cross - section from mie function
* Size distribution gamma
* Lwc and effective radius data is from LES


## initialization

In [31]:
import numpy as np
import scipy.io as sio
import shdom
import glob
import os 

## Parsering files

 ### Mie .scat file

In [32]:
# Open a file
f = open("../vadim/mie_tables/polydisperse/Water_672nm.scat", "r+")

scat_lines = f.readlines()

# Close opend file
f.close()

wavelength = float(scat_lines[1].split()[0])

# effective variance is chosen to be 0.1 - selecting the closest greater equal to 0.1
var_eff_vec = np.array(scat_lines[6].split()[:3], dtype=float)
var_eff_vec = np.linspace(var_eff_vec[1], var_eff_vec[2], int(var_eff_vec[0]))
var_eff_par = round(var_eff_vec[ var_eff_vec >= 0.1 ][0], 4)

num_var_eff = len(var_eff_vec)
num_r_eff   = int(scat_lines[5].split()[0])

g              = np.zeros(num_r_eff)
r_eff_scat     = np.zeros(num_r_eff)
var_eff_scat   = np.zeros(num_r_eff)
cross_sec_scat = np.zeros(num_r_eff)

ind1 = -1
ind2 = -1

for line_ind, line in enumerate(scat_lines[9:]):
    line_arr = np.array(line.split())
    if (line_arr[-1] == 'Nleg'):
        if (float(line.split()[1]) == var_eff_par ):
            ind1                += 1
            r_eff_scat[ind1]     = float(line.split()[0])
            var_eff_scat[ind1]   = float(line.split()[1])
            cross_sec_scat[ind1] = float(line.split()[2])
        
    elif (float(line_arr[0]) == 1.) and (ind2 < ind1):
        ind2   += 1
        g[ind2] = float(line.split()[1]) / 3.


### LWC text file

In [33]:
# Open a file
f = open("../synthetic_cloud_fields/Tamars_data/rico20sn1024x1024x65.txt", "r+")

lwc_lines = f.readlines()

# Close opend file
f.close()

nx = int(lwc_lines[3].split()[0])
ny = int(lwc_lines[3].split()[1])
nz = int(lwc_lines[3].split()[2])

x_min = 0.0
y_min = 0.0
z_min = float(lwc_lines[4].split()[2])
z_max = float(lwc_lines[4].split()[-1])

cloud_field = np.zeros((nx, ny, nz))
reff_field = np.zeros((nx, ny, nz))
lwc_field = np.zeros((nx, ny, nz))


for line in lwc_lines[5:]:
    [ indx, indy, indz ] = np.array(line.split()[:3], dtype=int) - 1
    [ lwc_s, r_eff_s ]   = np.array(line.split()[3:], dtype=float)
    reff_field[ indx, indy, indz ] = r_eff_s
    lwc_field[ indx, indy, indz ] = lwc_s
    
    if r_eff_s < r_eff_scat.min():
        cloud_field[ indx, indy, indz ] = 0.
    else:
        nearst_reff_ind = np.argmin(abs(r_eff_scat - r_eff_s))
        cloud_field[ indx, indy, indz ] = lwc_s * cross_sec_scat[nearst_reff_ind]


In [4]:
filename_out = 'rico20sn1024x1024x65_water_672nm.mat'

sio.savemat(filename_out, {'beta' : cloud_field})
print( 'extinction calculation is saved to ' + filename_out )

extinction calculation is saved to rico20sn1024x1024x65_water_672nm.mat


Use mayavi to plot the medium

In [5]:
from mayavi import mlab

In [16]:
def load_MAT_FILE(mat_path):
    if os.path.exists(mat_path):
        print("loading the 3D mat from: {}".format(mat_path))
    else:
        print("{} not exists".format(mat_path))    
    
    matrix3D = sio.loadmat(mat_path)
    return matrix3D

def viz3D(cloud_field):
        
    nx,ny,nz = cloud_field.shape
    dx, dy, dz = (1,1,1)
    
    xgrid = np.linspace(0, nx - 1,nx)
    ygrid = np.linspace(0, ny - 1,ny)
    zgrid = np.linspace(0, nz - 1,nz) 
    X, Y, Z = np.meshgrid(xgrid, ygrid, zgrid, indexing='ij')
    figh = mlab.gcf()
    src = mlab.pipeline.scalar_field(X, Y, Z, cloud_field)
    src.spacing = [dx, dy, dz]
    src.update_image_data = True 
    
    isosurface = mlab.pipeline.iso_surface(src, contours=[0.1*cloud_field.max(),\
                                                          0.2*cloud_field.max(),\
                                                          0.3*cloud_field.max(),\
                                                          0.4*cloud_field.max(),\
                                                          0.5*cloud_field.max(),\
                                                          0.6*cloud_field.max(),\
                                                          0.7*cloud_field.max(),\
                                                          0.8*cloud_field.max(),\
                                                          0.9*cloud_field.max(),\
                                                          ],opacity=0.9)
    mlab.pipeline.volume(isosurface, figure=figh)
    color_bar = mlab.colorbar(title="volume", orientation='vertical', nb_labels=5)

    mlab.outline(figure=figh,color = (1, 1, 1))  # box around data axes
    mlab.orientation_axes(figure=figh)
    mlab.axes(figure=figh, xlabel="x (km)", ylabel="y (km)", zlabel="z (km)")    
    mlab.show()    

Plot the whole medium

In [17]:
nx,ny,nz = cloud_field.shape
xgrid = np.linspace(0, nx - 1,nx)
ygrid = np.linspace(0, ny - 1,ny)
zgrid = np.linspace(0, nz - 1,nz) 
X, Y, Z = np.meshgrid(xgrid, ygrid, zgrid, indexing='ij')

viz3D(cloud_field)


Cut the medium and creat many sub-mediums and save the txt files

In [52]:
# Field = reff_field
# Field = lwc_field

# mediume parameters:
dx,dy,dz=(1e-3*20,1e-3*20,1e-3*40) # in km
zgrid = np.linspace(z_min, z_max-dz ,nz)

# cut function
def cut_medium(Field,type_='volume',zgrid=[0]):
    if(type_ is 'lwc'):
        lwc0 = Field[76:95, 134:153, 38:57]
        z0 = zgrid[38:57]
        
        lwc1 = Field[76:105, 124:153, 28:57]
        z1 = zgrid[28:57]
            
        lwc2 = Field[61:120, 109:168, 6:65]
        z2 = zgrid[6:65]
     
        lwc3 = Field[51:130, 99:178, :]
        z3 = zgrid
        
        lwc4 = Field[41:140, 89:188,:]
        z4 = zgrid
        
        lwc5 = Field[16:135, 79:198, :]
        z5 = zgrid
        
        lwc6 = Field[0:150, 0:250, :]
        z6 = zgrid
        
        med_list = [lwc0,lwc1,lwc2,lwc3,lwc4,lwc5,lwc6]
        zrange_list = [z0,z1,z2,z3,z4,z5,z6]
        
    if(type_ is 'reff'):
        reff0 = Field[76:95, 134:153, 38:57]
        z0 = zgrid[38:57]
        
        reff1 = Field[76:105, 124:153, 28:57]
        z1 = zgrid[28:57]
        
        reff2 = Field[61:120, 109:168, 6:65]
        z2 = zgrid[6:65]
        
        reff3 = Field[51:130, 99:178, :]
        z3 = zgrid
        
        reff4 = Field[41:140, 89:188,:]
        z4 = zgrid
        
        reff5 = Field[16:135, 79:198, :]
        z5 = zgrid
        
        reff6 = Field[0:150, 0:250, :]
        z6 = zgrid
        
        med_list = [reff0,reff1,reff2,reff3,reff4,reff5,reff6]
        zrange_list = [z0,z1,z2,z3,z4,z5,z6]
    
    med_names = [type_+str(i) for i in range(7)]
    
    for med_name,med in zip(med_names,med_list):
        print("The shape  of {} is {}".format(med_name,med.shape))

    return med_list, zrange_list, med_names

# main
print("The shapes befor padding")
lwc_list, zrange_list, lwc_names = cut_medium(lwc_field,'lwc',zgrid)
reff_list, zrange_list_test, reff_names = cut_medium(reff_field,'reff',zgrid)
u = [i==j for i,j in zip(zrange_list,zrange_list_test)]
for i in u:
    for j in i:
        assert j ,"Problem with indexes ranges in func. cut_medium."


savetxt = True

# fixed padding:
npad = ((1, 1), (1, 1), (1, 1))

for lwc_name,lwc, reff,zrange in zip(lwc_names,lwc_list,reff_list,zrange_list):
    #viz3D(lwc)
    
    # do the padding here
    tmpvol_lwc = np.pad(lwc, pad_width=npad, mode='constant', constant_values=0.0)
    tmpvol_reff = np.pad(reff, pad_width=npad, mode='constant', constant_values=0.0)
    print("The shape of {}, after padding is {}".format(lwc_name,tmpvol_lwc.shape))
    
    # set cloud bottom to zero:
    zds = np.linspace(0, (max(zrange)+dz) - (min(zrange)-dz) ,tmpvol_lwc.shape[2]) 
    print("The z range of {}, befor padding is [{},{}]".format(lwc_name,min(zrange),max(zrange)))
    print("The z range of {}, after padding is [{},{}]".format(lwc_name,zds.min(),zds.max()))
    if(savetxt):
        file_name = "tamar_cvpr_rico{}x{}x{}.txt"\
            .format(tmpvol_lwc.shape[0],tmpvol_lwc.shape[1],tmpvol_lwc.shape[2])
        # create the txt files:
        comment_line = "tamar cvpr"
        np.savetxt(file_name, X=np.array([tmpvol_lwc.shape]), fmt='%d', header=comment_line)
        f = open(file_name, 'ab') 
        
        
        
        
        np.savetxt(f, X=np.concatenate((np.array([dx, dy]), zds)).reshape(1,-1), fmt='%2.3f')
        
        nx, ny, nz = tmpvol_lwc.shape
        lwc = tmpvol_lwc
        reff = tmpvol_reff
        
        y, x, z = np.meshgrid(range(ny), range(nx), range(nz))
        data = np.vstack((x.ravel(), y.ravel(), z.ravel(), lwc.ravel(), reff.ravel())).T
        np.savetxt(f, X=data, fmt='%d %d %d %.5f %.3f')        
        f.close()  



The shapes befor padding
The shape  of lwc0 is (19, 19, 19)
The shape  of lwc1 is (29, 29, 29)
The shape  of lwc2 is (59, 59, 59)
The shape  of lwc3 is (79, 79, 65)
The shape  of lwc4 is (99, 99, 65)
The shape  of lwc5 is (119, 119, 65)
The shape  of lwc6 is (150, 250, 65)
The shape  of reff0 is (19, 19, 19)
The shape  of reff1 is (29, 29, 29)
The shape  of reff2 is (59, 59, 59)
The shape  of reff3 is (79, 79, 65)
The shape  of reff4 is (99, 99, 65)
The shape  of reff5 is (119, 119, 65)
The shape  of reff6 is (150, 250, 65)
The shape of lwc0, after padding is (21, 21, 21)
The z range of lwc0, befor padding is [1.52,2.24]
The z range of lwc0, after padding is [0.0,0.8000000000000003]
The shape of lwc1, after padding is (31, 31, 31)
The z range of lwc1, befor padding is [1.12,2.24]
The z range of lwc1, after padding is [0.0,1.2000000000000002]
The shape of lwc2, after padding is (61, 61, 61)
The z range of lwc2, befor padding is [0.24,2.56]
The z range of lwc2, after padding is [0.0,2.4]

load and visualize the saved txt and evaluate

In [57]:


txt_files = sorted(glob.glob('./tamar_cvpr_rico*.txt'))
txt_files_names = [os.path.split(i)[-1] for i in txt_files]
# Mie scattering for water droplets
mie = shdom.MiePolydisperse()
mie.read_table(file_path='/home/vhold/pyshdom/vadim/mie_tables/polydisperse/Water_672nm.scat')

for file in txt_files_names:
    mlab.figure(size=(600, 600))
    droplets = shdom.MicrophysicalScatterer()
    droplets.load_from_csv(file, veff=0.1)
    droplets.add_mie(mie)
    
    Grid_bounding_box = droplets.bounding_box
    Grid_shape = droplets.grid.shape
    xgrid = np.linspace(Grid_bounding_box.xmin, Grid_bounding_box.xmax,Grid_shape[0])
    ygrid = np.linspace(Grid_bounding_box.ymin, Grid_bounding_box.ymax,Grid_shape[1])
    zgrid = np.linspace(Grid_bounding_box.zmin, Grid_bounding_box.zmax,Grid_shape[2])     

    X, Y, Z = np.meshgrid(xgrid, ygrid, zgrid, indexing='ij')
    LWC_MAT = droplets.lwc.data
    RE_MAT = droplets.reff.data  
    print("-- current lwc size is {}".format(LWC_MAT.shape))
    nx, ny, nz = LWC_MAT.shape
    
    dx = droplets.lwc.grid.dx
    dy = droplets.lwc.grid.dy
    dz = (Grid_bounding_box.zmax- Grid_bounding_box.zmin)/(nz-1)
    print("-- current dx,dy,dz are {},{},{}".format(dx,dy,dz))
    viz3D(LWC_MAT)
    viz3D(RE_MAT)

Reading mie table from file: /home/vhold/pyshdom/vadim/mie_tables/polydisperse/Water_672nm.scat
-- current lwc size is (101, 101, 67)
-- current dx,dy,dz are 0.019999999552965164,0.019999999552965164,0.04000000158945719
-- current lwc size is (121, 121, 67)
-- current dx,dy,dz are 0.019999999552965164,0.019999999552965164,0.04000000158945719
-- current lwc size is (152, 252, 67)
-- current dx,dy,dz are 0.019999999552965164,0.019999999552965164,0.04000000158945719
-- current lwc size is (21, 21, 21)
-- current dx,dy,dz are 0.019999999552965164,0.019999999552965164,0.04000000059604645
-- current lwc size is (31, 31, 31)
-- current dx,dy,dz are 0.019999999552965164,0.019999999552965164,0.04000000158945719
-- current lwc size is (61, 61, 61)
-- current dx,dy,dz are 0.019999999552965164,0.019999999552965164,0.04000000158945719
-- current lwc size is (81, 81, 67)
-- current dx,dy,dz are 0.019999999552965164,0.019999999552965164,0.04000000158945719


In [48]:
zgrid = np.linspace(z_min, z_max-dz ,nz)
zgrid

array([0.  , 0.04, 0.08, 0.12, 0.16, 0.2 , 0.24, 0.28, 0.32, 0.36, 0.4 ,
       0.44, 0.48, 0.52, 0.56, 0.6 , 0.64, 0.68, 0.72, 0.76, 0.8 , 0.84,
       0.88, 0.92, 0.96, 1.  , 1.04, 1.08, 1.12, 1.16, 1.2 , 1.24, 1.28,
       1.32, 1.36, 1.4 , 1.44, 1.48, 1.52, 1.56, 1.6 , 1.64, 1.68, 1.72,
       1.76, 1.8 , 1.84, 1.88, 1.92, 1.96, 2.  , 2.04, 2.08, 2.12, 2.16,
       2.2 , 2.24, 2.28, 2.32, 2.36, 2.4 , 2.44, 2.48, 2.52, 2.56])

In [49]:
zgrid.shape

(65,)

# Tamars new medium:

In [34]:
"""
Cropped cloud from JPL, saved in rico20sn1024x1024x65_water_672nm.mat

With the following assumptions:
- Effective varince = 0.1031
- Albedo =1
- Wavelete 672[ nm ]
- Size distribution gamma
- Cross - section from mie function
- Lwc and effective radius data is from LES

extinction units are km^(-1)]nx spacing in km = 0.02
y spacing in km = 0.02
z spacing in km = 0.04
cloud indexs (xs:xe,ys:ye,:) = (14:65,1:50,:)
"""

dx,dy,dz=(1e-3*20,1e-3*20,1e-3*40) # in km
zgrid = np.linspace(z_min, z_max-dz ,nz)

# cut function
def cut_medium(Field,type_='volume',zgrid=[0]):
    if(type_ is 'lwc'):
        #188:287,875:1024,:
        lwc = Field[187:287,874:1024,:]
        z = zgrid
        
        med = lwc
        
    if(type_ is 'reff'):
        reff = Field[187:287,874:1024,:]
        z = zgrid
        
        med = reff
    
    
        print("The shape  of {} is {}".format(type_,med.shape))

    return med, z

# main
print("The shapes befor padding")
lwc, zrange = cut_medium(lwc_field,'lwc',zgrid)
reff, zrange = cut_medium(reff_field,'reff',zgrid)




The shapes befor padding
The shape  of reff is (100, 150, 65)


In [35]:
savetxt = True

# fixed padding:
npad = ((1, 1), (1, 1), (1, 1))

# viz3D(lwc)

# do the padding here
tmpvol_lwc = np.pad(lwc, pad_width=npad, mode='constant', constant_values=0.0)
tmpvol_reff = np.pad(reff, pad_width=npad, mode='constant', constant_values=0.0)

# set cloud bottom to zero:
zds = np.linspace(0, (max(zrange)+dz) - (min(zrange)-dz) ,tmpvol_lwc.shape[2]) 
print("The z range befor padding is [{},{}]".format(min(zrange),max(zrange)))
print("The z range after padding is [{},{}]".format(zds.min(),zds.max()))
if(savetxt):
    file_name = "tamar_cvpr_rico{}x{}x{}.txt"\
        .format(tmpvol_lwc.shape[0],tmpvol_lwc.shape[1],tmpvol_lwc.shape[2])
    # create the txt files:
    comment_line = "tamar cvpr"
    np.savetxt(file_name, X=np.array([tmpvol_lwc.shape]), fmt='%d', header=comment_line)
    f = open(file_name, 'ab') 




    np.savetxt(f, X=np.concatenate((np.array([dx, dy]), zds)).reshape(1,-1), fmt='%2.3f')

    nx, ny, nz = tmpvol_lwc.shape
    lwc = tmpvol_lwc
    reff = tmpvol_reff

    y, x, z = np.meshgrid(range(ny), range(nx), range(nz))
    data = np.vstack((x.ravel(), y.ravel(), z.ravel(), lwc.ravel(), reff.ravel())).T
    np.savetxt(f, X=data, fmt='%d %d %d %.5f %.3f')        
    f.close()  


The z range befor padding is [0.0,2.56]
The z range after padding is [0.0,2.64]


In [10]:
mat_path = "../synthetic_cloud_fields/Tamars_data/cloud_field2.mat"
matrix3D = load_MAT_FILE(mat_path)
matrix3D['beta_cf3']

loading the 3D mat from: ../synthetic_cloud_fields/Tamars_data/cloud_field2.mat


array([[[0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        ...,
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.]],

       [[0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        ...,
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.]],

       [[0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        ...,
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.]],

       ...,

       [[0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        ...,
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0.

In [29]:
f = [1,2,3,4,5,6,7]
f[0:7] # in matlab it is f(1:7)

[1, 2, 3, 4, 5, 6, 7]

In [30]:
287-188-1

98